使用K近邻算法对breast cancer数据进行评估，原则：近朱者赤近墨者黑

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import time
%matplotlib inline

In [ ]:
先对数据进行预览，主要目的是将diagnosis的类别数据转为数字

In [2]:
data = pd.read_csv("F:\Machine Learning\ML\Data\Datasets\Breast-Cancer//train.csv ")
data.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [10]:
def loadData(fileName):
    '''
    加载breast cancer数据集
    :param fileName:要加载的数据集路径
    :return: list形式的数据集及标记
    '''
    print('数据读取')
    # 存放数据及标记的list
    dataArr = []; labelArr = []
    # 打开文件
    fr = open(fileName, 'r')
    # 将文件按行读取
    lines=fr.readlines()
    for line in lines[1:]:
        # 对每一行数据按切割福','进行切割，返回字段列表
        curLine = line.strip().split(',')
        if curLine[1]=='M':
            labelArr.append(-1)
        else:
            labelArr.append(1)
        #存放标记
        dataArr.append([float(num) for num in curLine[2:]])
    #返回data和label
    return dataArr, labelArr

In [11]:
def calcDist(x1, x2):
    '''
    计算两个样本点向量之间的距离
    使用的是欧氏距离，即 样本点每个元素相减的平方，再求和，再开方
    欧式举例公式这里不方便写，可以百度或谷歌欧式距离（也称欧几里得距离）
    :param x1:向量1
    :param x2:向量2
    :return:向量之间的欧式距离
    '''
    return np.sqrt(np.sum(np.square(x1 - x2)))
    #曼哈顿距离计算公式
    # return np.sum(x1 - x2)

In [12]:
def getClosest(trainDataMat, trainLabelMat, x, topK):
    '''
    预测样本x的标记。
    获取方式通过找到与样本x最近的topK个点，并查看它们的标签。
    查找里面占某类标签最多的那类标签
    :param trainDataMat:训练集数据集
    :param trainLabelMat:训练集标签集
    :param x:要预测的样本x
    :param topK:选择参考最邻近样本的数目（样本数目的选择关系到正确率）
    :return:预测的标记
    '''
    #建立一个存放向量x与每个训练集中样本距离的列表
    #列表的长度为训练集的长度，distList[i]表示x与训练集中第
    ## i个样本的距离
    distList = [0] * len(trainLabelMat)
    #遍历训练集中所有的样本点，计算与x的距离
    for i in range(len(trainDataMat)):
        #获取训练集中当前样本的向量
        x1 = trainDataMat[i]
        #计算向量x与训练集样本x的距离
        curDist = calcDist(x1, x)
        #将距离放入对应的列表位置中
        distList[i] = curDist
    #对距离列表进行排序
    #argsort：函数将数组的值从小到大排序后，并按照其相对应的索引值输出
    #例如：
    #   >>> x = np.array([3, 1, 2])
    #   >>> np.argsort(x)
    #   array([1, 2, 0])
    #返回的是列表中从小到大的元素索引值，对于我们这种需要查找最小距离的情况来说很合适
    #array返回的是整个索引值列表，我们通过[:topK]取列表中前topL个放入list中。
    topKList = np.argsort(np.array(distList))[:topK]        #升序排序
    #建立一个长度时的列表，用于选择数量最多的标记
    #3.2.4提到了分类决策使用的是投票表决，topK个标记每人有一票，在数组中每个标记代表的位置中投入
    #自己对应的地方，随后进行唱票选择最高票的标记
    labelList = [0] * 2
    #对topK个索引进行遍历
    for index in topKList:
        #trainLabelMat[index]：在训练集标签中寻找topK元素索引对应的标记
        #int(trainLabelMat[index])：将标记转换为int（实际上已经是int了，但是不int的话，报错）
        #labelList[int(trainLabelMat[index])]：找到标记在labelList中对应的位置
        #最后加1，表示投了一票
        labelList[int(trainLabelMat[index])] += 1
    #max(labelList)：找到选票箱中票数最多的票数值
    #labelList.index(max(labelList))：再根据最大值在列表中找到该值对应的索引，等同于预测的标记
    return labelList.index(max(labelList))


In [13]:
def test(trainDataArr, trainLabelArr, testDataArr, testLabelArr, topK):
    '''
    测试正确率
    :param trainDataArr:训练集数据集
    :param trainLabelArr: 训练集标记
    :param testDataArr: 测试集数据集
    :param testLabelArr: 测试集标记
    :param topK: 选择多少个邻近点参考
    :return: 正确率
    '''
    print('start test')
    #将所有列表转换为矩阵形式，方便运算
    trainDataMat = np.mat(trainDataArr); trainLabelMat = np.mat(trainLabelArr).T
    testDataMat = np.mat(testDataArr); testLabelMat = np.mat(testLabelArr).T
    #错误值计数
    errorCnt = 0
    #遍历测试集，对每个测试集样本进行测试
    #和return也要相应的更换注释行
    for i in range(len(testDataMat)):
        print('test %d:%d'%(i, len(trainDataArr)))
        #读取测试集当前测试样本的向量
        x = testDataMat[i]
        #获取预测的标记
        y = getClosest(trainDataMat, trainLabelMat, x, topK)
        #如果预测标记与实际标记不符，错误值计数加1
        if y != testLabelMat[i]: errorCnt += 1
    #返回正确率
    return 1 - (errorCnt / len(testDataMat))

In [17]:
start = time.time()
#获取训练集
trainData, trainLabel = loadData('F:\Machine Learning\ML\Data\Datasets\Breast-Cancer//train.csv ')
#获取测试集
testData, testLabel = loadData('F:\Machine Learning\ML\Data\Datasets\Breast-Cancer//test.csv ')
#计算测试集正确率
accur = test(trainData, trainLabel, testData, testLabel, 10)
#打印正确率
print('accur is:%d'%(accur * 100), '%')
end = time.time()
#显示花费时间
print('time span:', end - start)

数据读取
数据读取
start test
test 0:469
test 1:469
test 2:469
test 3:469
test 4:469
test 5:469
test 6:469
test 7:469
test 8:469
test 9:469
test 10:469
test 11:469
test 12:469
test 13:469
test 14:469
test 15:469
test 16:469
test 17:469
test 18:469
test 19:469
test 20:469
test 21:469
test 22:469
test 23:469
test 24:469
test 25:469
test 26:469
test 27:469
test 28:469
test 29:469
test 30:469
test 31:469
test 32:469
test 33:469
test 34:469
test 35:469
test 36:469
test 37:469
test 38:469
test 39:469
test 40:469
test 41:469
test 42:469
test 43:469
test 44:469
test 45:469
test 46:469
test 47:469
test 48:469
test 49:469
test 50:469
test 51:469
test 52:469
test 53:469
test 54:469
test 55:469
test 56:469
test 57:469
test 58:469
test 59:469
test 60:469
test 61:469
test 62:469
test 63:469
test 64:469
test 65:469
test 66:469
test 67:469
test 68:469
test 69:469
test 70:469
test 71:469
test 72:469
test 73:469
test 74:469
test 75:469
test 76:469
test 77:469
test 78:469
test 79:469
test 80:469
test 81:469
test 